# LangGraph Message Trimming with `MessagesState` and `RemoveMessage`

This notebook demonstrates how **LangGraph manages long-running
conversations** using:

- `MessagesState` for built-in message accumulation
- `add_messages` as the default reducer
- `RemoveMessage` for controlled memory trimming
- Conditional routing to maintain a bounded context window

The goal is to preserve recent conversational context
while preventing unbounded memory growth.


In [ ]:
import getpass
import os
from langgraph.graph import START, END, StateGraph, add_messages, MessagesState
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, RemoveMessage
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from collections.abc import Sequence
from typing import Literal, Annotated

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Message Accumulation with `add_messages`

`add_messages` is a reducer that merges:
- Existing messages in state
- Newly returned messages from a node

It ensures conversation history is **appended**, not overwritten.


In [ ]:
my_list = add_messages([HumanMessage("Hi! I'm Oscar. "), 
                        AIMessage("Hey, Oscar! How can I assist you?")], 
                        [HumanMessage("Could you summarize today's news?")])

## Chat Model Initialization

A lightweight deterministic model is used
to keep the focus on graph behavior rather than generation quality.


In [ ]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'}},
    ) 


## Defining Graph Nodes

Each node:
- Receives the full message history
- Prints the current state for inspection
- Returns new messages to be merged into state


In [ ]:
def ask_question(state: MessagesState) -> MessagesState:
    
    print(f"\n-------> ENTERING ask_question:")
    for i in state["messages"]:
        i.pretty_print()
    
    question = "What is your question?"
    print(question)
    
    return MessagesState(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def chatbot(state: MessagesState) -> MessagesState:
    
    print(f"\n-------> ENTERING chatbot:")
    for i in state["messages"]:
        i.pretty_print()
    
    response = chat.invoke(state["messages"])
    response.pretty_print()
    
    return MessagesState(messages = [response])

In [ ]:
def ask_another_question(state: MessagesState) -> MessagesState:
    
    print(f"\n-------> ENTERING ask_another_question:")
    for i in state["messages"]:
        i.pretty_print()
    
    question = "Would you like to ask one more question (yes/no)?"
    print(question)
    
    return MessagesState(messages = [AIMessage(question), HumanMessage(input())])

## Trimming Conversation History

To prevent unlimited growth of conversation history,
older messages are removed using `RemoveMessage`.

This implementation keeps only the **last 5 messages**
and removes everything before them.


In [ ]:
def trim_messages(state: MessagesState) -> MessagesState:
    print(f"\n-------> ENTERING trim_messages:")
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:-5]]
    
    return MessagesState(messages = remove_messages)

## Conditional Routing

After each interaction, the user decides whether
the conversation should continue.

If the user answers `"yes"`, old messages are trimmed
and the conversation continues.


In [ ]:
def routing_function(state: MessagesState) -> Literal["trim_messages", "__end__"]:
    
    if state["messages"][-1].content == "yes":
        return "trim_messages"
    else:
        return "__end__"

## Building the Graph

The graph combines:
- Conversational nodes
- Conditional looping
- Memory trimming


In [ ]:
graph = StateGraph(MessagesState)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)
graph.add_node("trim_messages", trim_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question", 
                            path = routing_function)
graph.add_edge("trim_messages", "ask_question")

In [ ]:
graph_compiled = graph.compile()

In [ ]:
graph_compiled

## Executing the Graph

The graph starts with an empty message history.
Messages are accumulated and trimmed automatically.


In [ ]:
graph_compiled.invoke(MessagesState(messages = []))

## `RemoveMessage` Usage Example

The following example demonstrates how `RemoveMessage`
can be used to selectively remove earlier messages
from a conversation history.


In [ ]:
my_list = add_messages([AIMessage("What is your question?"), 
                        HumanMessage("Could you tell me a grook by Piet Hein?"),
                        AIMessage("Certainly! Here's a well-known grook by Piet Hein..."),
                        AIMessage("Would you like to ask one more question?"),
                        HumanMessage("yes"),
                        AIMessage("What is your question?"),
                        HumanMessage("Where was the poet born?"),
                        AIMessage("Piet Hein was born in Copenhagen, Denmark, on December 16, 1905."),
                        AIMessage("Would you like to ask one more question?")],
                       [HumanMessage("yes")]
                      )

In [ ]:
my_list

In [ ]:
remvove_message = [RemoveMessage(id = i.id) for i in my_list[:5]]

In [ ]:
remvove_message

## Summary

This notebook demonstrated:

- Using `MessagesState` for built-in message accumulation
- Trimming conversation history with `RemoveMessage`
- Implementing sliding-window memory
- Conditional routing with stateful conversations
- Managing long-running chat interactions safely

This pattern mirrors real-world conversational agents,
where memory must be controlled without losing relevance.
